In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

### DCP

In [ ]:
def my_cumulative_sum(input_image, r):
    height, width = input_image.shape
    cumulative_sum = np.zeros_like(input_image)

    # Calculate cumulative sum for columns
    cumulative = np.cumsum(input_image, axis=0)
    cumulative_sum[:r+1, :] = cumulative[r:2*r+1, :]
    cumulative_sum[r+1:height-r, :] = cumulative[2*r+1:height, :] - cumulative[:height-2*r-1, :]
    cumulative_sum[height-r:, :] = np.tile(cumulative[height-1, :], (r, 1)) - cumulative[height-2*r-1:height-r-1, :]

    # Calculate cumulative sum for rows
    cumulative = np.cumsum(cumulative_sum, axis=1)
    cumulative_sum[:, :r+1] = cumulative[:, r:2*r+1]
    cumulative_sum[:, r+1:width-r] = cumulative[:, 2*r+1:width] - cumulative[:, :width-2*r-1]
    cumulative_sum[:, width-r:] = np.tile(cumulative[:, width-1][:, np.newaxis], (1, r)) - cumulative[:, width-2*r-1:width-r-1]

    return cumulative_sum

def my_minfilter(I, window_size):
    I_new = I.copy()
    height, width = I.shape

    for i in range(height):
        for j in range(width):
            i_down = max(0, i - window_size)
            i_up = min(height, i + window_size + 1)
            j_down = max(0, j - window_size)
            j_up = min(width, j + window_size + 1)
            
            I_new[i, j] = np.min(I[i_down:i_up, j_down:j_up])

    return I_new

def my_darkchannel(I, window_size):
    height, width, _ = I.shape
    dark_channel = np.ones((height, width))

    for i in range(height):
        for j in range(width):
            dark_channel[i, j] = np.min(I[i, j, :])

    min_dark_channel = my_minfilter(dark_channel, window_size)
    return min_dark_channel

def my_estimateA(I, dark_channel):
    A = np.zeros((1, 1, 3))
    height, width = dark_channel.shape

    points_number = round(width * height * 0.001)

    for _ in range(points_number):
        # brightest_points = np.max(dark_channel)
        i, j = np.unravel_index(np.argmax(dark_channel), dark_channel.shape)
        
        dark_channel[i, j] = 0

        if np.mean(I[i, j, :]) > np.mean(A):
            A[0, 0, :] = (A[0, 0, :] + I[i, j, :]) / 2

    return A

def my_guidedfilter(guide_image, I, radius, smooth_parameter):
    height, width = guide_image.shape
    N = my_cumulative_sum(np.ones((height, width)), radius)

    mean_guide = my_cumulative_sum(guide_image, radius) / N
    mean_I = my_cumulative_sum(I, radius) / N
    mean_IG = my_cumulative_sum(guide_image * I, radius) / N
    cov_IG = mean_IG - mean_guide * mean_I
    mean_II = my_cumulative_sum(guide_image * guide_image, radius) / N
    var_I = mean_II - mean_guide * mean_guide

    a = cov_IG / (var_I + smooth_parameter)
    b = mean_I - a * mean_guide

    mean_a = my_cumulative_sum(a, radius) / N
    mean_b = my_cumulative_sum(b, radius) / N

    q = mean_a * guide_image + mean_b
    return q


In [ ]:
def run_DCP(I, threshold=0.01):

    # Get dark channel
    window_size = 7
    dark_channel = my_darkchannel(I, window_size)
    dark_channel = dark_channel / 255.0

    # Calculate atmospheric light A
    I = I.astype(np.float32) / 255.0
    A = my_estimateA(I, dark_channel)

    # Calculate transmission matrix t(x)
    w = 0.95
    t = 1 - w * dark_channel / np.mean(A)

    I_gray = cv2.cvtColor(I, cv2.COLOR_RGB2GRAY)

    t1 = my_guidedfilter(I_gray, t, 135, 0.0002)
    t2 = my_guidedfilter(t1, t1, 7, 0.03)

    # t_threshold = 0.01
    t = np.maximum(t2, threshold)

    # Recover haze-free image
    K = 0.2
    defog_image = np.zeros_like(I)

    epsilon = 0.000001
    for i in range(3):
        defog_image[:, :, i] = (
            (I[:, :, i] - A[0, 0, i])
            / np.minimum(1, t * np.maximum(K / (np.abs(I[:, :, i]+epsilon) - A[0, 0, i]), 1))
        ) + A[0, 0, i]

    defog_image = defog_image * 1.3
    defog_image = np.clip(defog_image, 0, 1)
    
    return defog_image

In [ ]:
images = [
    "./images/4_CLAHE.jpg"
]

for image in images:
    inp = cv2.imread(image)
    res = run_DCP(inp, 0.05)
    
    plt.figure(figsize=(18, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(inp, cv2.COLOR_BGR2RGB))
    plt.title(f"Original Image")
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
    plt.title(f"After DCP")
    plt.show()

### CLAHE

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def my_clip_histogram(hist, sum_pixel_bin, clip_limit, subpart_x, subpart_y):
    """
    Clip histogram to limit contrast enhancement
    """
    for i in range(subpart_x):
        for j in range(subpart_y):
            # Calculate excess histogram values
            sum_excess = 0
            for nr in range(sum_pixel_bin):
                excess = hist[i, j, nr] - clip_limit
                if excess > 0:
                    sum_excess += excess

            # Redistribute excess histogram values
            bin_average = sum_excess / sum_pixel_bin
            upper = clip_limit - bin_average

            for nr in range(sum_pixel_bin):
                if hist[i, j, nr] > clip_limit:
                    hist[i, j, nr] = clip_limit
                else:
                    if hist[i, j, nr] > upper:
                        sum_excess += upper - hist[i, j, nr]
                        hist[i, j, nr] = clip_limit
                    else:
                        sum_excess -= bin_average
                        hist[i, j, nr] += bin_average

            # Distribute remaining excess
            if sum_excess > 0:
                step_size = max(1, int(1 + sum_excess / sum_pixel_bin))
                for nr in range(sum_pixel_bin):
                    sum_excess -= step_size
                    hist[i, j, nr] += step_size
                    if sum_excess < 1:
                        break

    return hist

def my_map_histogram(hist, min_pixel, max_pixel, sum_pixel_bins, sum_pixels, subpart_x, subpart_y):
    """
    Map histogram to output range
    """
    output = np.zeros((subpart_x, subpart_y, sum_pixel_bins))
    scale = (max_pixel - min_pixel) / sum_pixels

    for i in range(subpart_x):
        for j in range(subpart_y):
            cumsum = 0
            for nr in range(sum_pixel_bins):
                cumsum += hist[i, j, nr]
                output[i, j, nr] = min(min_pixel + cumsum * scale, max_pixel)

    return output

def my_adapthisteq(img_gray):
    """
    Adaptive Histogram Equalization implementation
    """
    # Ensure image is float type
    img_gray = img_gray.astype(np.float32)

    # Get image dimensions
    height, width = img_gray.shape

    # Get pixel value range
    min_pixel = np.min(img_gray)
    max_pixel = np.max(img_gray)

    # Image subdivision
    subpart_y = max(1, int(width / 100) - 2)
    subpart_x = max(1, int(height / 100) - 1)

    height_size = int(np.ceil(height / subpart_x))
    width_size = int(np.ceil(width / subpart_y))

    # Adjust image dimensions to ensure perfect division
    delta_y = subpart_x * height_size - height
    delta_x = subpart_y * width_size - width

    # Pad image
    temp_image = np.zeros((height + delta_y, width + delta_x), dtype=np.float32)
    temp_image[:height, :width] = img_gray

    new_width = width + delta_x
    new_height = height + delta_y
    sum_pixels = width_size * width_size

    # Pixel bin mapping
    sum_pixel_bins = 256
    
    # Create pixel bin with proper indexing
    pixel_bin = np.zeros_like(temp_image, dtype=np.int32)
    for m in range(temp_image.shape[0]):
        for n in range(temp_image.shape[1]):
            # Ensure value is within 0-255 range and add 1 for 1-based indexing
            pixel_bin[m, n] = min(max(0, int(temp_image[m, n])), 255) + 1

    # Create histogram
    hist = np.zeros((subpart_x, subpart_y, 256), dtype=np.float32)
    for i in range(subpart_x):
        for j in range(subpart_y):
            sub_img = pixel_bin[i * height_size:(i + 1) * height_size, 
                                 j * width_size:(j + 1) * width_size]
            hist[i, j, :], _ = np.histogram(sub_img, bins=256, range=(1, 257))

    # Clip histogram
    clip_limit = 2.5
    clip_limit = max(1, clip_limit * height_size * width_size / sum_pixel_bins)
    hist = my_clip_histogram(hist, sum_pixel_bins, clip_limit, subpart_x, subpart_y)

    # Map histogram
    map_output = my_map_histogram(hist, min_pixel, max_pixel, sum_pixel_bins, sum_pixels, subpart_x, subpart_y)

    # Bilinear interpolation for output
    output = np.zeros_like(pixel_bin, dtype=np.float32)
    y_i = 0
    for i in range(subpart_x + 1):
        # Handle row boundaries
        if i == 0:
            sub_y = int(height_size / 2)
            y_up = 0
            y_bottom = 0
        elif i == subpart_x:
            sub_y = int(height_size / 2)
            y_up = subpart_x - 1
            y_bottom = subpart_x - 1
        else:
            sub_y = height_size
            y_up = i - 1
            y_bottom = i

        x_i = 0
        for j in range(subpart_y + 1):
            # Handle column boundaries
            if j == 0:
                sub_x = int(width_size / 2)
                x_left = 0
                x_right = 0
            elif j == subpart_y:
                sub_x = int(width_size / 2)
                x_left = subpart_y - 1
                x_right = subpart_y - 1
            else:
                sub_x = width_size
                x_left = j - 1
                x_right = j

            # Interpolation
            u_l = map_output[y_up, x_left, :]
            u_r = map_output[y_up, x_right, :]
            b_l = map_output[y_bottom, x_left, :]
            b_r = map_output[y_bottom, x_right, :]

            sub_image = pixel_bin[y_i:y_i + sub_y, x_i:x_i + sub_x]
            s_image = np.zeros_like(sub_image, dtype=np.float32)

            for m in range(sub_y):
                for n in range(sub_x):
                    val = sub_image[m, n] - 1  # Adjust index
                    s_image[m, n] = (
                        (sub_y - m) * ((sub_x - n) * u_l[val] + n * u_r[val]) +
                        m * ((sub_x - n) * b_l[val] + n * b_r[val])
                    ) / (sub_y * sub_x)

            output[y_i:y_i + sub_y, x_i:x_i + sub_x] = s_image
            x_i += sub_x

        y_i += sub_y

    # Crop back to original dimensions
    output = output[:height, :width]
    return output.astype(np.uint8)

def run_CLAHE(img):
    """
    Process multiple images using CLAHE
    """

    # Convert to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_img)

    # Apply adaptive histogram equalization to L channel
    l_enhanced = my_adapthisteq(l_channel)

    # Subtract 50 from L channel
    l_enhanced = np.clip(l_enhanced.astype(np.float32) - 50, 0, 255).astype(np.uint8)

    # Merge channels back
    enhanced_lab_img = cv2.merge([l_enhanced, a_channel, b_channel])

    # Convert back to BGR
    enhanced_img = cv2.cvtColor(enhanced_lab_img, cv2.COLOR_LAB2BGR)

    # Increase brightness
    enhanced_img = np.clip(1.35 * enhanced_img, 0, 255).astype(np.uint8)

    return enhanced_img

In [ ]:
def CLAHE_DCP(images, weighted=False, only_DCP_threshold=0.1, CLAHE_DCP_threshold=0.05):
    for image_path in images:
        img = cv2.imread(image_path)

        after_CLAHE = run_CLAHE(img)
        after_DCP = run_DCP(after_CLAHE*0.5 + img*0.5, threshold=CLAHE_DCP_threshold) if weighted else run_DCP(after_CLAHE, threshold=CLAHE_DCP_threshold)
    
        only_DCP = run_DCP(img, threshold=only_DCP_threshold)

        plt.figure(figsize=(20, 10))

        plt.subplot(1, 4, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"original image")

        plt.subplot(1, 4, 2)
        plt.imshow(cv2.cvtColor(after_CLAHE, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"only CLAHE")

        plt.subplot(1, 4, 3)
        plt.imshow(cv2.cvtColor(after_DCP, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"CLAHE + DCP")

        plt.subplot(1, 4, 4)
        plt.imshow(cv2.cvtColor(only_DCP, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"only DCP")

        plt.show()

In [ ]:
folder = "./images/new_images"

images = [
    f"{folder}/{i}.jpg" for i in range(2, 3)
    ]

CLAHE_DCP(images, only_DCP_threshold=0.1, CLAHE_DCP_threshold=0.001)

In [ ]:
folder = "./images"

images = [
    f"{folder}/carrrr.png",
    f"{folder}/delhi.png",
    f"{folder}/foggy_lil.png",
    f"{folder}/test_d1.png",
    f"{folder}/test_paper.png",
    f"{folder}/test1.png"
]

CLAHE_DCP(images, weighted=False)

### Homomorphic

In [ ]:
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift

In [ ]:
def My_homofilter(I_mean):
    # Step 1: Take logarithm and perform Fourier transform
    I_log = np.log(I_mean + 1)
    I_fft = fft2(I_log)
    I_fft = fftshift(I_fft)

    # Step 2: Frequency domain Gaussian high-pass filtering
    
    # Gaussian filter parameters
    L = 0.3
    H = 1.8
    C = 2
    # Cutoff frequency D0
    D0 = 1

    # Create mask
    height, width = I_mean.shape
    y, x = np.ogrid[:height, :width]
    center_y, center_x = height // 2, width // 2
    D = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    mask = (H - L) * (1 - np.exp(C * (-D / D0))) + L  # Gaussian homomorphic filter

    # Apply mask
    I_fft_gauss = mask * I_fft
    
    # Step 3: Inverse Fourier transform and take exponent
    I_fft_gauss = ifftshift(I_fft_gauss)
    
    I_ifft = ifft2(I_fft_gauss)
    # Take exponent to restore original image
    I_gray_defog = np.real(np.exp(I_ifft) - 1)
    
    return I_gray_defog


def run_Homomorphic(I):
    start_time = time.time()
    
    # Homomorphic filtering
    I_mean = np.mean(I, axis=2)
    # I_mean = I_mean.astype(np.float64) / 255.0    
    I_mean = np.clip(I_mean, 1e-6, None).astype(np.float64) / 255.0


    I_gray_defog = My_homofilter(I_mean)

    max_pixel = np.max(I_gray_defog)
    I_gray_defog = (I_gray_defog - np.min(I_gray_defog)) / (max_pixel - np.min(I_gray_defog))

    # Apply homomorphic filter result to each channel
    I_defog = np.zeros_like(I, dtype=np.float64)
    for i in range(3):
        I_defog[:,:,i] = (I[:,:,i].astype(np.float64) * I_gray_defog) / I_mean

    max_pixel = np.max(I_defog)
    min_pixel = np.min(I_defog)
    I_defog = (I_defog - min_pixel) / (max_pixel - min_pixel)
    
    # Enhance contrast
    I_defog = 1.35 * I_defog
    
    # normalize the image to [0, 255]
    max_pixel = np.max(I_defog)
    I_defog = ((255/max_pixel) * I_defog).astype(np.uint8)

    end_time = time.time()
    # print(f"Execution time: {end_time - start_time} seconds")
    
    return I_defog

In [ ]:
def Homo_DCP(images, weighted=False):
    for image_path in images:
        img = cv2.imread(image_path)

        after_Homomorphic = run_Homomorphic(img)
        after_DCP = run_DCP(after_Homomorphic*0.5 + img*0.5) if weighted else run_DCP(after_Homomorphic)
    
        only_DCP = run_DCP(img)

        plt.figure(figsize=(20, 10))

        plt.subplot(1, 4, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"original image")

        plt.subplot(1, 4, 2)
        plt.imshow(cv2.cvtColor(after_Homomorphic, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"only Homomorphic")

        plt.subplot(1, 4, 3)
        plt.imshow(cv2.cvtColor(after_DCP, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"Homomorphic + DCP")

        plt.subplot(1, 4, 4)
        plt.imshow(cv2.cvtColor(only_DCP, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"only DCP")

        plt.show()

In [ ]:
folder = "./images"

images = [
    f"{folder}/foggy_lil.png",
    f"{folder}/carrrr.png",
    f"{folder}/delhi.png",
    f"{folder}/test_d1.png",
    f"{folder}/test_paper.png",
    f"{folder}/test1.png"
]

Homo_DCP(images)

In [ ]:
folder = "./images/new_images"

images = [
    f"{folder}/{i}.jpg" for i in range(1, 9)
    ]

Homo_DCP(images)